In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
# import tushare as ts
# import QUANTAXIS as QA
import talib as ta
import datetime, time
from dateutil.relativedelta import *
from pandas.tseries.offsets import *
from tqdm import tqdm_notebook

In [3]:
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
plt.style.use('ggplot')

crsp fiscal resample

In [4]:
def calc_exf(df):
    ret = df.ret_p1
    dret = df.dret_p1
    exf = df.mv - df.mv.shift(1) * df.retx_p1
    exf_to_mv = exf / df.mv.shift(1)
    
    car1 = ((ret[::-1]).rolling(1, min_periods=1).apply(np.prod, raw=True)[::-1]) - \
        ((dret[::-1]).rolling(1, min_periods=1).apply(np.prod, raw=True)[::-1])
    car3 = ((ret[::-1]).rolling(3, min_periods=1).apply(np.prod, raw=True)[::-1]) - \
        ((dret[::-1]).rolling(3, min_periods=1).apply(np.prod, raw=True)[::-1])
    car5 = ((ret[::-1]).rolling(5, min_periods=1).apply(np.prod, raw=True)[::-1]) - \
        ((dret[::-1]).rolling(5, min_periods=1).apply(np.prod, raw=True)[::-1])
    
    wr5 = ((ret[::-1]).rolling(5, min_periods=1).apply(np.prod, raw=True)[::-1]) / \
        ((dret[::-1]).rolling(5, min_periods=1).apply(np.prod, raw=True)[::-1])
    
    ret_n1 = ((ret[::-1]).rolling(1, min_periods=1).apply(np.prod, raw=True)[::-1])
    ret_n2 = ((ret[::-1]).rolling(2, min_periods=1).apply(np.prod, raw=True)[::-1])
    ret_n3 = ((ret[::-1]).rolling(3, min_periods=1).apply(np.prod, raw=True)[::-1])
    ret_n4 = ((ret[::-1]).rolling(4, min_periods=1).apply(np.prod, raw=True)[::-1])
    ret_n5 = ((ret[::-1]).rolling(5, min_periods=1).apply(np.prod, raw=True)[::-1])
    ret_n6 = ((ret[::-1]).rolling(6, min_periods=1).apply(np.prod, raw=True)[::-1])
    ret_n7 = ((ret[::-1]).rolling(7, min_periods=1).apply(np.prod, raw=True)[::-1])
    ret_n8 = ((ret[::-1]).rolling(8, min_periods=1).apply(np.prod, raw=True)[::-1])
    
    res = pd.DataFrame({
        'date':df.date,
        'mv':df.mv,
        'retx_p1':df.retx_p1,
        'exf':exf,
        'exf_to_mv':exf_to_mv,
        'car1':car1.shift(-1),
        'car3':car3.shift(-1),
        'car5':car5.shift(-1),
        'wr5':wr5.shift(-1),
        'ret_n1':ret_n1.shift(-1),
        'ret_n2':ret_n2.shift(-1),
        'ret_n3':ret_n3.shift(-1),
        'ret_n4':ret_n4.shift(-1),
        'ret_n5':ret_n5.shift(-1),
        'ret_n6':ret_n6.shift(-1),
        'ret_n7':ret_n7.shift(-1),
        'ret_n8':ret_n8.shift(-1)
    })
    return res

In [19]:
crsp_fa = pd.read_csv('data/crsp_fa.zip',
                     parse_dates=['date'], infer_datetime_format=True)

In [18]:
# crsp_fa.head()

,date,mv,retx_p1,exf,exf_to_mv,car1,car3,car5,wr5,ret_n1,ret_n2,ret_n3,ret_n4,ret_n5,ret_n6,ret_n7,ret_n8
permno,,,,,,,,,,,,,,,,,
10001,2001-06-30,29653.400,1.475001,NaN,NaN,0.608938,0.608938,0.608938,1.543873,1.728571,1.728571,1.728571,1.728571,1.728571,1.728571,1.728571,1.728571
10001,2007-06-30,42885.000,1.662972,-6427.765667,-2.167632e-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10002,1993-12-31,38612.125,1.430557,NaN,NaN,0.111624,0.626210,-0.335900,0.838203,1.039473,1.135239,2.228429,2.121083,1.740153,1.370505,1.534391,1.797741
10002,1994-12-31,39361.875,1.019418,-0.008577,-2.221426e-07,-0.281362,0.412748,-0.743871,0.639306,1.092129,2.143805,2.040536,1.674071,1.318460,1.476123,1.729472,2.581610
10002,1995-12-31,42028.000,1.066668,41.937460,1.065433e-03,0.705720,-0.096200,-0.527141,0.719418,1.962960,1.868403,1.532852,1.207239,1.351602,1.583579,2.363833,3.730554


查看firm-years数量

ps:因为流通股数的单位默认是1000, 故mv_adj只需要大于50000即可

In [8]:
# crsp_fa[crsp_fa.mv_adj >= 5e+4].shape[0]

145087

In [6]:
crsp_fa[(crsp_fa.mv_adj * crsp_fa.retx_p1) >= 5e+4].shape[0]

143115

In [20]:
# crsp_fa = crsp_fa[crsp_fa.mv_adj >= 5e+4]
crsp_fa = crsp_fa[crsp_fa.mv_adj * crsp_fa.retx_p1 >= 5e+4]

In [21]:
tic = time.perf_counter()
crsp_fa = crsp_fa.set_index('permno').groupby('permno').apply(calc_exf)
toc = time.perf_counter()
print(toc-tic)

274.9635354000002



exf_to_mv : $EXF_{t}/MV_{t-1}$

ret_n1 : 接下来1年的return


In [15]:
from scipy.stats import mstats

In [20]:
crsp_fa.head()

,date,mv,retx_p1,exf,exf_to_mv,car1,car3,car5,wr5,ret_n1,ret_n2,ret_n3,ret_n4,ret_n5,ret_n6,ret_n7,ret_n8
permno,,,,,,,,,,,,,,,,,
10002,1993-12-31,38612.125,1.430557,NaN,NaN,0.111624,-0.337573,0.341395,0.341395,1.039473,1.135239,1.085200,2.130205,2.027591,1.663451,1.310096,1.466758
10002,1994-12-31,39361.875,1.019418,-0.008577,-2.221426e-07,-0.281362,0.121447,-0.897720,-0.897720,1.092129,1.043990,2.049311,1.950594,1.600282,1.260346,1.411059,1.653241
10002,1995-12-31,42028.000,1.066668,41.937460,1.065433e-03,-0.160509,0.462912,-0.522323,-0.522323,0.955922,1.876438,1.786048,1.465287,1.154027,1.292026,1.513779,2.259641
10002,1996-12-31,39026.000,0.928571,-0.001807,-4.298500e-08,0.705720,-0.096200,-0.527141,-0.527141,1.962960,1.868403,1.532852,1.207239,1.351602,1.583579,2.363833,3.730554
10002,1997-12-31,104027.000,1.884614,30478.045526,7.809677e-01,0.009172,-0.579293,-0.533391,-0.533391,0.951829,0.780888,0.615009,0.688553,0.806730,1.204218,1.900473,1.593407


winsorize

In [23]:
crsp_fa1 = crsp_fa.copy()
crsp_fa1 = crsp_fa1.dropna()
crsp_fa1 = crsp_fa1.reset_index().set_index('date')
crsp_fa1 = crsp_fa1.loc['1973':'2001']
# 可以自行调整limits大小
crsp_fa1.exf_to_mv = mstats.winsorize(crsp_fa1.exf_to_mv, limits=[0.01, 0.01])

per1 = np.linspace(0,1,11).tolist()
bins = crsp_fa1.describe(percentiles=per1).exf_to_mv.iloc[4:-1].tolist()
bins = [bins[0]-1] + bins[1:]
# bins = bins[:-1] + [bins[-1]-1]
crsp_fa1.groupby(pd.cut(crsp_fa1.exf_to_mv,bins, right=True)).mean().drop(['mv', 'retx_p1', 'exf'], axis=1)

,permno,exf_to_mv,car1,car3,car5,wr5,ret_n1,ret_n2,ret_n3,ret_n4,ret_n5,ret_n6,ret_n7,ret_n8
exf_to_mv,,,,,,,,,,,,,,
"(-2.6, -0.0866]",53439.240350,-0.606063,0.398202,1.212484,2.345176,2.108341,1.584700,2.142044,2.776512,3.456564,4.278142,5.020127,5.567750,6.129881
"(-0.0866, -0.0119]",50686.039534,-0.037091,0.083394,0.284154,0.511553,1.309796,1.233968,1.479246,1.747576,2.024728,2.332819,2.666043,2.945749,3.151697
"(-0.0119, -7.34e-07]",50333.030557,-0.003239,0.067816,0.231714,0.517096,1.278447,1.235700,1.488061,1.751765,2.068528,2.461320,2.812095,3.059330,3.341910
"(-7.34e-07, 0.000238]",50670.644548,0.000016,0.068277,0.246712,0.469779,1.252071,1.259924,1.540766,1.837972,2.184599,2.548247,2.949071,3.373061,3.760708
"(0.000238, 0.00344]",48427.826589,0.001640,0.072456,0.254098,0.474990,1.264916,1.244601,1.503542,1.780939,2.076803,2.417530,2.748845,3.073675,3.486183
"(0.00344, 0.00947]",49454.445766,0.006130,0.073030,0.261087,0.516970,1.278784,1.236977,1.506758,1.778707,2.110445,2.431147,2.754596,3.034281,3.344594
"(0.00947, 0.0225]",50582.515569,0.015018,0.084164,0.335354,0.567882,1.296827,1.246009,1.547444,1.850859,2.144401,2.463470,2.774823,3.075380,3.410781
"(0.0225, 0.0614]",51730.309540,0.037824,0.098421,0.363571,0.633644,1.327490,1.253913,1.563035,1.871390,2.191356,2.510429,2.818651,3.090205,3.346457
"(0.0614, 0.198]",53213.571778,0.116081,0.086305,0.356322,0.678484,1.318160,1.243062,1.543663,1.865286,2.185931,2.572636,2.913826,3.307033,3.729615


Significant issuance & Significant repurchase

EXF is greater than 5% of pre-issuance market capitalization

$EXF_t >= 0.05 * (1+rx_t) *MV_{t-1}$

equals

$EXF_t / MV_{t-1} >= 0.05 * (1+rx_t)$

In [22]:
crsp_fa1[crsp_fa1.exf_to_mv > (0.05 * crsp_fa1.retx_p1)].describe().loc[['count', 'mean']]

,mv,retx_p1,exf,exf_to_mv,car1,car3,car5,wr5,ret_n1,ret_n2,ret_n3,ret_n4,ret_n5,ret_n6,ret_n7,ret_n8
count,2.261600e+04,22616.000000,22616.000000,22616.000000,22616.000000,22616.000000,22616.000000,22616.000000,22616.000000,22616.00000,22616.000000,22616.000000,22616.000000,22616.00000,22616.000000,22616.000000
mean,1.601562e+06,1.319307,309343.184603,0.315463,0.073115,0.295459,0.513976,1.261769,1.216344,1.46644,1.717428,1.953259,2.202909,2.44901,2.716505,3.007423


In [26]:
crsp_fa1[crsp_fa1.exf_to_mv <= (-0.05 * crsp_fa1.retx_p1)].describe().loc[['count', 'mean']]

,mv,retx_p1,exf,exf_to_mv,car1,car3,car5,wr5,ret_n1,ret_n2,ret_n3,ret_n4,ret_n5,ret_n6,ret_n7,ret_n8
count,1.298300e+04,12983.000000,12983.000000,12983.000000,12983.000000,12983.000000,12983.000000,12983.000000,12983.00000,12983.00000,12983.000000,12983.00000,12983.00000,12983.000000,12983.000000,12983.000000
mean,1.234386e+06,1.646566,-268720.910432,-0.528973,0.307569,0.906926,1.694573,1.836437,1.48047,1.91045,2.388611,2.88963,3.46422,3.998087,4.386622,4.768259


In [27]:
crsp_fa1[(crsp_fa1.exf_to_mv >(-0.05 * crsp_fa1.retx_p1)) & \
         (crsp_fa1.exf_to_mv <= (0.05 * crsp_fa1.retx_p1))].describe().loc[['count', 'mean']]

,mv,retx_p1,exf,exf_to_mv,car1,car3,car5,wr5,ret_n1,ret_n2,ret_n3,ret_n4,ret_n5,ret_n6,ret_n7,ret_n8
count,7.469700e+04,74697.000000,74697.000000,74697.000000,74697.000000,74697.000000,74697.000000,74697.000000,74697.000000,74697.000000,74697.000000,74697.000000,74697.000000,74697.000000,74697.000000,74697.00000
mean,2.130842e+06,1.158785,-2645.247907,0.005464,0.065972,0.229159,0.425221,1.244086,1.218782,1.445427,1.672068,1.918506,2.172896,2.415509,2.655951,2.90981


long Significant repurchase,  short Significant issuance

In [32]:
crsp_fa1[crsp_fa1.exf_to_mv <= (-0.05 * crsp_fa1.retx_p1)].describe().loc[['mean']] - \
crsp_fa1[crsp_fa1.exf_to_mv > (0.05 * crsp_fa1.retx_p1)].describe().loc[['mean']]

,mv,retx_p1,exf,exf_to_mv,car1,car3,car5,wr5,ret_n1,ret_n2,ret_n3,ret_n4,ret_n5,ret_n6,ret_n7,ret_n8
mean,-367175.786559,0.32726,-578064.095035,-0.844436,0.234454,0.611467,1.180597,0.574668,0.264126,0.444009,0.671183,0.936371,1.26131,1.549077,1.670117,1.760836


查看样本的个数

In [21]:
crsp_fa1.groupby(pd.cut(crsp_fa1.exf_to_mv,bins, right=True)).count().exf_to_mv

exf_to_mv
(-2.734, -0.0777]       11030
(-0.0777, -0.0126]      11030
(-0.0126, -8.13e-07]    11029
(-8.13e-07, 0.00044]    11030
(0.00044, 0.00412]      11029
(0.00412, 0.0106]       11030
(0.0106, 0.0239]        11029
(0.0239, 0.0616]        11030
(0.0616, 0.193]         11029
(0.193, 1.259]          11030
Name: exf_to_mv, dtype: int64

In [163]:
result = crsp_fa.copy()

In [167]:
writer = pd.ExcelWriter('data/sample_fiscal.xlsx', datetime_format='YYYY-MM-DD')
crsp_fa[crsp_fa.permno==77418].to_excel(writer, index=False)
writer.save()